### This is Natural Language Processing Project in which Natural Language Toolkit is used and base on Extraction Summarization

## Preparing the data. Here data is scraping from wikipedia

In [101]:
import bs4 as BeautifulSoup  #use to fetcha the articles text
import urllib.request        #use for connectiong to the page and retrieving the HTML
import re
# Fetching the content from the URL
fetched_data = urllib.request.urlopen('https://en.wikipedia.org/wiki/Brahma_Kumaris')

article_read = fetched_data.read()

# Parsing the URL content and storing in a variable
article_parsed = BeautifulSoup.BeautifulSoup(article_read,'lxml')

# Returning <p> tags
paragraphs = article_parsed.find_all('p')

article_content = ''

# Looping through the paragraphs and adding them to the variable
for p in paragraphs:  
    article_content += p.text



## Processing the data

In [54]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
def _create_dictionary_table(text_string) -> dict:
   
    # Removing stop words
    stop_words = set(stopwords.words("english"))
    
    words = word_tokenize(text_string)
    
    # Reducing words to their root form
    stem = PorterStemmer()
    
    # Creating dictionary for the word frequency table
    frequency_table = dict()
    for wd in words:
        wd = stem.stem(wd)
        if wd in stop_words:
            continue
        if wd in frequency_table:
            frequency_table[wd] += 1
        else:
            frequency_table[wd] = 1

    return frequency_table

In [85]:
len(formatted_article_content)

17067

In [87]:
frequency_table=_create_dictionary_table(formatted_article_content)


## Tokenizing the article into sentences

In [83]:
from nltk.tokenize import word_tokenize, sent_tokenize
sentences = sent_tokenize(article_content)
(sentences)

126

## Finding the weighted frequencies of the sentences

In [65]:
def _calculate_sentence_scores(sentences, frequency_table) -> dict:   

    # Algorithm for scoring a sentence by its words
    sentence_weight = dict()

    for sentence in sentences:
        sentence_wordcount = (len(word_tokenize(sentence)))
        sentence_wordcount_without_stop_words = 0
        for word_weight in frequency_table:
            if word_weight in sentence.lower():
                sentence_wordcount_without_stop_words += 1
                if sentence[:7] in sentence_weight:
                    sentence_weight[sentence[:7]] += frequency_table[word_weight]
                else:
                    sentence_weight[sentence[:7]] = frequency_table[word_weight]

        sentence_weight[sentence[:7]] = sentence_weight[sentence[:7]] /        sentence_wordcount_without_stop_words
      
    return sentence_weight

In [82]:
sentence_scores=_calculate_sentence_scores(sentences, frequency_table)

## Calculating the threshold of the sentences

In [67]:
def _calculate_average_score(sentence_weight) -> int:
   
    # Calculating the average score for the sentences
    sum_values = 0
    for entry in sentence_weight:
        sum_values += sentence_weight[entry]

    # Getting sentence average value from source text
    average_score = (sum_values / len(sentence_weight))

    return average_score


In [68]:
threshold = _calculate_average_score(sentence_scores)
threshold

4.152466325906582

## Function to generate a summary from article

In [69]:
def _get_article_summary(sentences, sentence_weight, threshold):
    sentence_counter = 0
    article_summary = ''

    for sentence in sentences:
        if sentence[:7] in sentence_weight and sentence_weight[sentence[:7]] >= (threshold):
            article_summary += " " + sentence
            sentence_counter += 1

    return article_summary

In [99]:
#producing the summary
summary_results = _get_article_summary(sentences, sentence_scores, 1.95* threshold)
print(summary_results)

 He reported what he said were a series of visions and other transcendental experiences that commenced around 1935 and became the basis for the discourses. He reported the rewriting of the revelatory messages (Murlis) by the Brahma Kumari.


In [100]:
len(summary_results)

239